<a href="https://colab.research.google.com/github/aveerdram/NLP/blob/main/%D0%98%D0%9B%D0%90%D0%99%D0%9D_%D1%82%D0%B5%D1%81%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание

Создайте модель, обрабатывающую фрагмент текста и определяющую
какой вид продукции в нём содержится.

Выполненное задание пришлите ссылкой на Google Collab.
Датасет поделите на 80% / 20% - обучающая/тестовая выборки.

Используйте библиотеку PyTorch

Датасет: https://axe.inline-ltd.ru/data/meatinfo.csv

Виды продукции (брать только виды продукции, для которых в датасете есть не менее 500 примеров):

* Баранина
* Ягнятина
* Индейка
* Говядина
* Свинина
* Кура
* Цыплено
* Гусь
* Буйволятина
* Оленина
* Конина
* Телятина
* Кролик
* Утка
* Куропатка
* Перепел
* Глухарь
* Страус
* Заяц
* Кенгуру
* Изюбр
* Кабан
* Коза
* Косуля
* Лось
* Марал
* Медвежатина
* Бобер
* Цесарка
* Нутрия
* Рябчик
* Тетерев
* Фазан
* Як


Примеры входных текстов и видов продукции:

1. Набор для бульона свиной Набор для бульона свиной, в наличии, 76р/кг. -> Свинина

2. Мясо премиум Предлагаем котлетное мясо мраморной говядины. -> Говядина

3. спинка цб -> Цыпленок

Проверьте вашу модель на образцах:

1. Говядина блочная 2 сорт в наличии ООО "АгроСоюз" реализует блочную говядину 2 сорт (80/20)
2. Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.

3. Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:

4. Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.

## Начало работы с датасетом

In [ ]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 41.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=830eec96ab820032cb0adf6bfd060d5181f72b0a1b68eab407868b3e012a0705
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import pandas as pd
import numpy as np
from sklearn import decomposition

import warnings
warnings.filterwarnings("ignore")

import seaborn as sns
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = (10,8)

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)

In [ ]:
import pandas as pd
df_raw = pd.read_csv("meatinfo.csv", sep=';', on_bad_lines='skip')
df_raw

,text,mtype
0,12 частей баранина 12 частей баранина,Баранина
1,"Баранина, 12 частей, зам. цена 260 руб.",Баранина
2,"Баранина, 12 частей, зам. цена 315 руб.",Баранина
3,"Баранина, 12 частей, охл.",Баранина
4,"Баранина, 12 частей, охл. цена 220 руб.",Баранина
...,...,...
17888,"Ягнятина, шея, бк",Ягнятина
17889,Язык ягненка (н.зеландия) Отварные языки ягнят с низким содержанием жира и богатые белком замечательно подходят для ...,Ягнятина
17890,"Ягнятина, язык, зачищ. цена 100 руб.",Ягнятина
17891,"Як, задние части, 1 категория цена 550 руб.",Як


In [ ]:
df = df_raw.copy()

## Виды продукции, для которых в датасете есть не менее 500 примеров

In [ ]:
classes = ['Баранина',
 'Ягнятина',
 'Индейка',
 'Говядина',
 'Свинина',
 'Кура',
 'Цыпленок',
 'Гусь',
 'Буйволятина',
 'Оленина',
 'Конина',
 'Телятина',
 'Кролик',
 'Утка',
 'Куропатка',
 'Перепел',
 'Глухарь',
 'Страус',
 'Заяц',
 'Кенгуру',
 'Изюбр',
 'Кабан',
 'Коза',
 'Косуля',
 'Лось',
 'Марал',
 'Медвежатина',
 'Бобер',
 'Цесарка',
 'Нутрия',
 'Рябчик',
 'Тетерев',
 'Фазан',
 'Як']

Оставим только те классы, частота встречи которых в датасете не меньше 500.

In [ ]:
from collections import Counter

cnt = Counter(df['mtype'])

cnt = dict(cnt)

result = {key: value for key, value in cnt.items() if value > 500}

result

{'Баранина': 1116,
 'Индейка': 1337,
 'Говядина': 8422,
 'Свинина': 3050,
 'Кура': 1571,
 'Цыпленок': 942}

In [ ]:
def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

my_tags = []

for val in result.values():
  my_tags.append(get_key(result, val))

my_tags

['Баранина', 'Индейка', 'Говядина', 'Свинина', 'Кура', 'Цыпленок']

In [ ]:
for i in range(len(df)):
  if df['mtype'][i] not in result:
    df = df.drop(index=[i])

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df['mtype'].unique()

array(['Баранина', 'Индейка', 'Говядина', 'Свинина', 'Кура', 'Цыпленок'],
      dtype=object)

In [ ]:
df

,text,mtype
0,12 частей баранина 12 частей баранина,Баранина
1,"Баранина, 12 частей, зам. цена 260 руб.",Баранина
2,"Баранина, 12 частей, зам. цена 315 руб.",Баранина
3,"Баранина, 12 частей, охл.",Баранина
4,"Баранина, 12 частей, охл. цена 220 руб.",Баранина
...,...,...
16433,"Цыпленок, четвертина, задняя",Цыпленок
16434,"Цыпленок, четвертина, задняя цена 100 руб.",Цыпленок
16435,"ЦБ Шеи п/ф Свеженка ГОСТ зам пак Шеи куриные ПОЗЦ Свеженка ГОСТ, заморозка, пакет",Цыпленок
16436,"Цыпленок, шея, без кожи",Цыпленок


## Предобработка данных

In [ ]:
import re
import string
import nltk
nltk.download("punkt")
from nltk import word_tokenize
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()
stopwords = set(stopwords.words('russian'))

In [ ]:
def clean_text(text):

    text = text.lower() # приводим к нижнему регистру
    text = re.sub(r"\[(.*?)\]", "", text)  # удаляем слова в квадратных скобках как [+300 chars
    text = re.sub(r"\s+", " ", text)  # удаляем множественный пробел, табуляцию и перенос строки
    text = ''.join(["" if t.isdigit() else t for t in text])
    text = re.sub(r'[^\w\s]', '', text)  # удаляем знаки препинания
    text = re.sub(r"\w+…|…", "", text)  # удаляем многоточие
    text = re.sub(r"(?<=\w)-(?=\w)", " ", text) # заменяем дефис на пробел
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # удаляем пунктуацию

    text = morph.parse(text)[0].normal_form

    return text

In [ ]:
df["tokens"] = df["text"].map(lambda x: clean_text(x))

# удаляем пустые значения и сохраняем два столбца: тексты и токены
df = df.loc[df.tokens.map(lambda x: len(x) > 0), ["text", "tokens", 'mtype']]

# удаляем дубликаты строк
_, idx = np.unique(df["tokens"], return_index=True)
df = df.iloc[idx, :]

docs = df["text"].values
tokenized_docs = df["tokens"].values

In [ ]:
print(f"Исходные данные: {df_raw.shape}")
print(f"Данные после предобработки: {df.shape}")
df.head(10)

Исходные данные: (17893, 2)
Данные после предобработки: (11641, 3)


,text,tokens,mtype
3385,220 ₽ мясо голов говяжьих Продам мясо голов говяжьих со щекой 220 с ндс,мясо голов говяжьих продам мясо голов говяжьих со щекой с ндс,Говядина
2492,3 вида карбоната Натуральный карбонат,вида карбоната натуральный карбонат,Свинина
3294,* Говядина из Сербии. Сухое мясо! Говядина из Сербии. Прямые поставки.,говядина из сербии сухое мясо говядина из сербии прямые поставка,Говядина
3379,"2 Говядина Мякоть Новосибирская Мясная Компания - один из лидеров рынка поставок свежей Алтайской говядины, более 10...",говядина мякоть новосибирская мясная компания один из лидеров рынка поставок свежей алтайской говядины более лет ...,Говядина
1564,!!! Грудинка Св. Ивр (дели) 248 р. На складе в г. Наро-Фоминск Грудинка св. ИВР (Дели) по 248 р.,грудинка св ивр дели р на складе в г нарофоминск грудинка св ивр дели по р,Свинина
1565,:) грудинка свин нк 162 р грудинка свин нк,грудинка свин нк р грудинка свин нк,Свинина
5459,", коровы, охл 162.00 Говядина, полутуши, 1 категория, коровы, охл, Белоруссия, подвес, склад, от 16 тонн\nб/н 162.00...",коровы охл говядина полутуши категория коровы охл белоруссия подвес склад от тонн бн руб предоплата доставка в...,Говядина
7550,"... машины ЦБ Продаю три машины ЦБ 1 сорт вес куры 1,7-2,2 кг. РБ .Приход на склад 37 неделя.Общий пакет.",машины цб продаю три машины цб сорт вес куры кг рб приход на склад неделяобщий пакет,Кура
5399,"охлажденное, 162.00 руб компания ООО “ МЖК ПРОМТОРГ” реализует на постоянной основе мясо говядины в полутушах,95...",охлажденное руб компания ооо мжк промторг реализует на постоянной основе мясо говядины в полутушах корова охлажде...,Говядина
3381,"2 сорт / ГОСТ / сухое мясо / 288 р Говядина 2 сорт, постность 80/20\nСухое мясо, подходит под все виды переработки.",сорт гост сухое мясо р говядина сорт постность сухое мясо подходит под все виды переработка,Говядина


## Модели

### Naive Bayes Classifier

In [ ]:
X = df['tokens']
y = df['mtype']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

In [ ]:
%%time
nb.fit(X_train, y_train)

CPU times: user 438 ms, sys: 1.81 ms, total: 440 ms
Wall time: 475 ms


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [ ]:
%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

CPU times: user 55.8 ms, sys: 0 ns, total: 55.8 ms
Wall time: 56.3 ms


In [ ]:
y_pred[0]

'Свинина'

In [ ]:
print(X_test.iloc[0], y_test.iloc[0], y_pred[0])

окорок свиной бк  рубкг продам окорок свиной бк гост текущей выработки выработан из охлажденного мяса свинины  категории окорок  с мясным орехом без голяшки цена  рубкг Свинина Свинина


In [ ]:
from sklearn.metrics import accuracy_score

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

accuracy 0.8063546586517819
              precision    recall  f1-score   support

    Баранина       0.94      0.15      0.26       115
     Индейка       0.77      1.00      0.87       981
    Говядина       0.99      0.51      0.67       136
     Свинина       0.63      0.80      0.71       295
        Кура       0.97      0.89      0.93       616
    Цыпленок       1.00      0.15      0.26       186

    accuracy                           0.81      2329
   macro avg       0.88      0.58      0.62      2329
weighted avg       0.84      0.81      0.77      2329



### Linear Support Vector Machine

In [ ]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])

In [ ]:
%%time
sgd.fit(X_train, y_train)

CPU times: user 508 ms, sys: 56.1 ms, total: 564 ms
Wall time: 1.02 s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf',
                 SGDClassifier(alpha=0.001, max_iter=5, random_state=42,
                               tol=None))])

In [ ]:
%%time
y_pred = sgd.predict(X_test)

CPU times: user 95.4 ms, sys: 875 µs, total: 96.2 ms
Wall time: 119 ms


In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.9330184628595964
              precision    recall  f1-score   support

    Баранина       0.99      0.93      0.96       115
     Индейка       0.96      1.00      0.98       981
    Говядина       0.98      0.95      0.97       136
     Свинина       0.80      0.79      0.80       295
        Кура       0.99      0.98      0.99       616
    Цыпленок       0.72      0.65      0.69       186

    accuracy                           0.93      2329
   macro avg       0.91      0.88      0.90      2329
weighted avg       0.93      0.93      0.93      2329



### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])

In [ ]:
%%time
logreg.fit(X_train, y_train)

CPU times: user 3.32 s, sys: 3.88 s, total: 7.2 s
Wall time: 5.21 s


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=100000.0, n_jobs=1))])

In [ ]:
%%time
y_pred = logreg.predict(X_test)

CPU times: user 103 ms, sys: 19.2 ms, total: 122 ms
Wall time: 123 ms


In [ ]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred, target_names=my_tags))

accuracy 0.918419922713611
              precision    recall  f1-score   support

    Баранина       0.96      0.94      0.95       115
     Индейка       0.98      0.99      0.98       981
    Говядина       0.98      0.96      0.97       136
     Свинина       0.73      0.77      0.75       295
        Кура       0.98      0.97      0.98       616
    Цыпленок       0.62      0.58      0.60       186

    accuracy                           0.92      2329
   macro avg       0.88      0.87      0.87      2329
weighted avg       0.92      0.92      0.92      2329



**Вывод по моделям: метод опорных векторов показал наиболее высокие оценки качества работы модели.**

## Проверка на примерах

In [ ]:
sgd.predict(['Говядина блочная 2 сорт в наличии ООО "АгроСоюз" реализует блочную говядину 2 сорт (80/20)'])

array(['Говядина'], dtype='<U8')

In [ ]:
sgd.predict(['Свободный объем 8 тонн Самовывоз или доставка. Все подробности по телефону.'])

array(['Говядина'], dtype='<U8')

In [ ]:
sgd.predict(['Куриная разделка Продам кур и куриную разделку гост и халяль по хорошей цене .Тел:'])

array(['Кура'], dtype='<U8')

In [ ]:
sgd.predict(['Говяжью мукозу Продам говяжью мукозу в охл и замороженном виде. Есть объем.'])

array(['Говядина'], dtype='<U8')